In [ ]:
import os

import pandas as pd
import numpy as np
from dotenv import load_dotenv

from psycopg2 import sql, connect
from sqlalchemy import create_engine, text

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from train_test.general import train_test_bookies_comparison, display_train_test_bookies_comparison, train_test
from train_test.split import train_test_split_expanding_windows, train_test_split_sliding_windows
from feature_eng.odds import bookie_prediction, prob_by_bookies
from train_test.metrics import accuracy_fn, classwise_ECE_fn, f_mesure_fn, log_loss_fn, mse_loss_fn, precision_fn, recall_fn
from feature_eng.team_stats import goal_stats, elo_scores, glicko2_scores, trueskill_scores


In [ ]:
load_dotenv(".env")
load_dotenv("secrets.env")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

DB_TN_MATCHES_EURO = os.getenv('DB_TN_MATCHES_EURO')
DB_TN_MATCHES = os.getenv('DB_TN_MATCHES')
DB_TN_MATCHES_BIG5 = os.getenv('DB_TN_MATCHES_BIG5')
DB_TN_MATCHES_WC = os.getenv('DB_TN_MATCHES_WC')
DB_TN_MATCHES_FBREF = os.getenv('DB_TN_MATCHES_FBREF')